# 0. 개요

- 모든 아이템을 대상으로 train 하는게 아닌, 각 카테고리별로 train 시키고 그거를 합치는 방식으로 진행해보자

# 1. 라이브러리 로드 및 데이터 불러오기

## 1.1 라이브러리 로드 및 메모리 감소 함수 정의

In [2]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix('demo/data/agaricus.txt.train')
dtest = xgb.DMatrix('demo/data/agaricus.txt.test')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

XGBoostError: [13:31:47] C:\Jenkins\workspace\xgboost-win64_release_0.90\dmlc-core\src\io\local_filesys.cc:127: LocalFileSystem.ListDirectory demo/data error: No such process

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def write_record(features, params):
    record = open("record model and features.txt", 'a')
    record.write("\n")
    record.write(str(datetime.datetime.now())+"\n")

    check = 0
    for _ in features:
        check += 1
        if check % 5 == 0:
            record.write("\n")
        record.write(_+"  ")
    record.write("\n")
    for i  in params.items():
        record.write(str(i) + "\n")

    record.write('--------------------------------\n')
    record.close()

## 1.2 데이터 불러오기 및 pd.melt를 활용해 데이터 정렬

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

In [ ]:
train = pd.read_csv('inputs/sales_train_validation.csv')
train = pd.melt(train, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='target')

test = pd.read_csv('inputs/sample_submission.csv')
test2 = test[30490:]

test = test[:30490]
test = pd.melt(test, id_vars=['id'], var_name='d', value_name='target')
for i in range(1, 29):
    test = test.replace({f'F{i}': f'd_{1913+i}'})

test[['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'tmp']] = pd.DataFrame(test['id'].str.split('_').tolist())
del test['tmp']
test['store_id'] = test['state_id'] + '_' + test['store_id']
test['dept_id'] = test['cat_id'] + '_' + test['dept_id']
test['item_id'] = test['dept_id'] + '_' + test['item_id']

test = test[train.columns]

calendar = pd.read_csv('inputs/calendar.csv')

sell_prices = pd.read_csv('inputs/sell_prices.csv')

In [ ]:
test.head()

In [ ]:
test.tail()

# 2. 데이터 탐색

## 2.1 train

In [ ]:
train.head(2)

In [ ]:
# 아웃라이어를 제거해서 boxplot 더 잘보이게끔
train200 = train[train['target'] < 10]

### 2.1.1 target

In [ ]:
pd.set_option('display.max_rows', 419)
pd.DataFrame(train['target'].value_counts())

In [ ]:
plt.figure(figsize=(10,7))
display(
    train[train['target'] == 763],
    train[train['id'] == 'FOODS_3_090_CA_3_validation'],
    sns.distplot(train[train['id'] == 'FOODS_3_090_CA_3_validation']['target'])    
)

In [ ]:
plt.figure(figsize=(10,7))
display(
    train[train['target'] == 370],
    train[train['id'] == 'FOODS_3_318_CA_3_validation'],
    sns.distplot(train[train['id'] == 'FOODS_3_318_CA_3_validation']['target'])    
)

In [ ]:
train[train['target'] == 763]

In [ ]:
plt.figure(figsize=(10,7))
sns.distplot(train['target'])

### 2.1.2 id

In [ ]:
train.groupby('id')['target'].mean()

### 2.1.3 item_id

In [ ]:
train.groupby('item_id')['target'].mean()

### 2.1.4 dept_id

In [ ]:
train.groupby('dept_id')['target'].mean()

In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(train['dept_id'], train200['target'])

### 2.1.5 cat_id

In [ ]:
train.groupby('cat_id')['target'].mean()

### 2.1.6 store_id

In [ ]:
train.groupby('store_id')['target'].mean()

### 2.1.7 state_id

In [ ]:
train.groupby('state_id')['target'].mean()

### 2.1.8 d

In [ ]:
train.groupby('d')['target'].mean()

## 2.2 test

In [ ]:
test.head(2)

## 2.3 calendar

- 대부분 date 데이터고, event 및 snap(정부보조금 적용되는 날)에 집중해보자.
- event_name_2가 너무 적다. 우선을 빼고 모델 만들어 볼 것임.
- 이벤트 유무(is_event) 컬럼 만들 예정

In [ ]:
calendar.info()

In [ ]:
calendar = calendar.drop(['event_name_2', 'event_type_2'], axis=1)
calendar['is_event'] = calendar['event_name_1'].notna().astype('int8')
del calendar['wday']
del calendar['wm_yr_wk'] # 모든 주차에 인덱스 붙여놓음. 282개

In [ ]:
calendar['day'] = calendar['date'].astype('datetime64').dt.day
calendar['week'] = calendar['date'].astype('datetime64').dt.week


In [ ]:
calendar.head()

In [ ]:
train = train.merge(calendar, how='left')
test = test.merge(calendar, how='left')

In [ ]:
train.head()

In [ ]:
train['day'] = train['date'].astype('datetime64').dt.day
test['day'] = test['date'].astype('datetime64').dt.day

train['week'] = train['date'].astype('datetime64').dt.week
test['week'] = test['date'].astype('datetime64').dt.week

## 2.4 sell_prices

In [ ]:
sell_prices.info()

In [ ]:
sell_prices.nunique()

In [ ]:
for i in sell_prices['wm_yr_wk'].unique():
    print(i)

In [ ]:
sell_prices.groupby('wm_yr_wk')['sell_price'].mean().index

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,7))
sns.lineplot(range(len(sell_prices.groupby('wm_yr_wk')['sell_price'].mean())), list(sell_prices.groupby('wm_yr_wk')['sell_price'].mean()))

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def write_record(features, params):
    record = open("record model and features.txt", 'a')
    record.write("\n")
    record.write(str(datetime.datetime.now())+"\n")

    check = 0
    for _ in features:
        check += 1
        if check % 5 == 0:
            record.write("\n")
        record.write(_+"  ")
    record.write("\n")
    for i  in params.items():
        record.write(str(i) + "\n")

    record.write('--------------------------------\n')
    record.close()

In [ ]:
train = pd.read_csv('inputs/sales_train_validation.csv')
train = pd.melt(train, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='target')
train = reduce_mem_usage(train)

In [ ]:
test = pd.read_csv('inputs/sample_submission.csv')
test = test[:30490]
test = pd.melt(test, id_vars=['id'], var_name='d', value_name='target')
for i in range(1, 29):
    test = test.replace({f'F{i}': f'd_{1913+i}'})

test[['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'tmp']] = pd.DataFrame(test['id'].str.split('_').tolist())
del test['tmp']
test['store_id'] = test['state_id'] + '_' + test['store_id']
test['dept_id'] = test['cat_id'] + '_' + test['dept_id']
test['item_id'] = test['dept_id'] + '_' + test['item_id']

test = test[train.columns]
test = reduce_mem_usage(test)

## 3.2 calendar (date 데이터)

In [ ]:
calendar = pd.read_csv('inputs/calendar.csv')

calendar = calendar.drop(['event_name_2', 'event_type_2'], axis=1)
calendar['is_event'] = calendar['event_name_1'].notna().astype('int8')
del calendar['wday']  # weekday랑 똑같은 컬럼.
calendar['day'] = calendar['date'].astype('datetime64').dt.day
calendar['week'] = calendar['date'].astype('datetime64').dt.week

In [ ]:
train = train.merge(calendar, how='left')
test = test.merge(calendar, how='left')

## 3.3 sell_prices

In [ ]:
sell_prices = pd.read_csv('inputs/sell_prices.csv')

In [ ]:
train = train.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
test = test.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

## 3.4 라벨인코딩

In [ ]:
all_df = pd.concat([train, test])

# del all_df['date']


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in all_df.columns[all_df.dtypes == 'object']:
    if i == 'id' or i == 'date':
        continue
    all_df[i] = le.fit_transform(list(all_df[i]))



## 3.5 lag 데이터 제작

In [ ]:
all_df['lag_t28'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28))
all_df['lag_t29'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(29))
all_df['lag_t30'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(30))

## 3.6 이동평균 피처

In [ ]:
all_df['rolling_mean_t7'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(7).mean())
all_df['rolling_std_t7'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(7).std())

all_df['rolling_mean_t30'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(30).mean())
all_df['rolling_std_t30'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(30).std())

all_df['rolling_mean_t90'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(90).mean())
all_df['rolling_mean_t180'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28).rolling(180).mean())


## 3.7 price 통계량 피쳐

In [ ]:
all_df['lag_price_t1'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))


all_df['price_change_t1'] = (all_df['lag_price_t1'] - all_df['sell_price']) / (all_df['lag_price_t1'])
all_df['rolling_price_max_t365'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
all_df['price_change_t365'] = (all_df['rolling_price_max_t365'] - all_df['sell_price']) / (all_df['rolling_price_max_t365'])
all_df['rolling_price_std_t7'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
all_df['rolling_price_std_t30'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)

In [ ]:
all_df = reduce_mem_usage(all_df)

# 4. 모델 제작

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
# rf.fit(train2, np.log(train['target'] + 1))
# result = rf.predict(test2)

In [ ]:
# from lightgbm import LGBMRegressor
# # lgb = LGBMRegressor(num_leaves=2000, colsample_bytree=0.6, subsample=0.6, n_estimators=600, learning_rate=0.025, n_jobs=-1, device='gpu', max_bin = 63)
# lgb = LGBMRegressor(num_leaves=20, colsample_bytree=0.6, subsample=0.6, n_estimators=60, learning_rate=0.02, n_jobs=-1, device='cpu')

# lgb.fit(train, target)
# result = lgb.predict(test)

## 4.1 train set /  test set 선정

In [ ]:
import lightgbm as lgb
import gc

features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'weekday', 'event_name_1', 'event_type_1',  
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30']

# 나중에 합칠 때 필요해서 test에 선언
test = all_df[len(train):]

train_set_X = all_df[:len(train)]
train_set_y = train_set_X['target']

train_set_X = train_set_X[features]

# 테스트 셋
test_set = all_df[len(train):]
test_set = test_set[features]

del all_df

In [ ]:
x_train = data[data['date'] <= '2016-03-27']
y_train = x_train['demand']
x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
y_val = x_val['demand']
test = data[(data['date'] > '2016-04-24')]
del data
gc.collect()

# define random hyperparammeters
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75}

train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

del x_train, y_train

model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)
val_pred = model.predict(x_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_pred = model.predict(test[features])
test['target'] = y_pred


## 4.2 KFold - LGBM 모델

In [ ]:
# n_fold = 5
# folds = KFold(n_splits=5, shuffle=True)
# splits = folds.split(train_set_X, train_set_y)

# y_preds = np.zeros(test.shape[0])
# y_oof = np.zeros(train.shape[0])

# feature_importances = pd.DataFrame()
# feature_importances['feature'] = train_set_X.columns
# mean_score = []

# for fold_n, (train_index, valid_index) in enumerate(splits):
#     print('Fold:',fold_n+1)
    
#     X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
#     y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
#     lgb = LGBMRegressor(
#         boosting_type = 'gbdt',
#         num_leaves = 2000,
#         colsample_bytree = 0.8,
#         subsample = 0.8,
#         n_estimators = 2500,
#         learning_rate = 0.02,
#         n_jobs = -1,
#         device = 'gpu'
#     )
#     lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 50, verbose = True)

#     feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
#     y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
#     y_oof[valid_index] = y_pred_valid
    
#     val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    
#     print(f'val rmse score is {val_score}')
    
#     mean_score.append(val_score)
    
#     y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
#     del X_train, X_valid, y_train, y_valid

# print('mean rmse score over folds is',np.mean(mean_score))
# test['target'] = y_preds



## 4.3 TimeSeriesSplit - LGBM 모델

In [ ]:
params = lgb.get_params()

In [ ]:
# n_fold = 10
# folds = TimeSeriesSplit(n_splits=n_fold)
# splits = folds.split(train_set_X, train_set_y)

# y_preds = np.zeros(test.shape[0])
# y_oof = np.zeros(train.shape[0])

# feature_importances = pd.DataFrame()
# feature_importances['feature'] = train_set_X.columns
# mean_score = []

# for fold_n, (train_index, valid_index) in enumerate(splits):
#     print('Fold:',fold_n+1)
    
#     X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
#     y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
#     lgb = LGBMRegressor(
#         num_leaves = 1000,
#         colsample_bytree = 0.8,
#         subsample = 0.8,
#         n_estimators = 2500,
#         learning_rate = 0.01,
#         n_jobs = -1,
#         device = 'cpu'
#     )
    
#     lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 50, verbose = False)

#     feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
#     y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
#     y_oof[valid_index] = y_pred_valid
    
#     val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    
#     print(f'val rmse score is {val_score}')
    
#     mean_score.append(val_score)
    
#     y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
#     del X_train, X_valid, y_train, y_valid

# print('mean rmse score over folds is',np.mean(mean_score))

# test['target'] = y_preds

## 4.4 feature_importance 확인

In [ ]:
feature_importances

# 5. 예측 및 제출

In [ ]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'target']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'target').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('submissions/submission.csv', index = False)

In [ ]:
final.head()

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [ ]:
write_record(features, params)

In [ ]:
import joblib
# save model
joblib.dump(lgb, 'models/lgb.pkl')
# load model
# lgb = joblib.load('models/lgb.pkl')